This notebook contains code for calculating the tas anomaly of a given dataset. We use the Pangeo CMIP6 Public Dataset. A lot of this code was modified starting from code from the Pangeo Gallery,found [here](https://gallery.pangeo.io/repos/pangeo-gallery/cmip6/).

Additionally, the [xarray documentation](https://xarray.pydata.org/en/stable/) and [xarray-extras documentation](https://xarray-extras.readthedocs.io/en/latest/), along with the online documentation for any of the other used libraries are a useful resource.

In [1]:
from matplotlib import pyplot as plt
import xarray as xr
import numpy as np
import dask
from dask.diagnostics import progress
from tqdm.autonotebook import tqdm
import intake
import fsspec
import seaborn as sns
import pandas as pd
from tqdm.autonotebook import tqdm  # Fancy progress bars for our loops!
from dask_gateway import Gateway
from dask.distributed import Client
import cftime
import datetime


In [2]:
col = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")

After importing all relevant packages and using intake to open the pangeo cmip6 database, we construct our query. Note that this particular query requires all sources have all 5 experiments. This is due to the passing the .search() function the perameter `require_all_on`. Info at the [intake-esm docs](https://intake-esm.readthedocs.io/en/latest/api.html?highlight=search#intake_esm.core.esm_datastore.search).

In [1]:
expts = ['historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585']

query = dict(
    experiment_id = expts,
    variable_id = ['tas'],
    table_id = ['Amon'],
    source_id = ['ACCESS-ESM1-5','CESM2-WACCM', 'MPI-ESM1-2-HR'],
    member_id = 'r1i1p1f1'
)

# In the other two documents, I used col_subset instead of cat.
cat = col.search(require_all_on=['source_id'], **query)


print(col_subset.df.groupby("source_id")[
    ["experiment_id", "variable_id", "table_id"]
].nunique())

NameError: name 'col' is not defined

In [8]:
# Create an xarray dataset using our catalog.
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': False},cdf_  kwargs={'chunks': {}, 'decode_times': False})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


The next block of code is mainly for formatting all of the datasets into a more standard format. This is especially important if you want to make a graph using multiple

In [10]:
def drop_all_bounds(ds):
    """Drop coordinates like 'time_bounds' from datasets,
    which can lead to issues when merging."""
    drop_vars = [vname for vname in ds.coords if (('_bounds') in vname ) or ('_bnds') in vname]
    return ds.drop(drop_vars)

def open_dsets(df):
    """Open datasets from cloud storage and return xarray dataset."""
    dsets = [xr.open_zarr(fsspec.get_mapper(ds_url), consolidated=True).pipe(drop_all_bounds) for ds_url in df.zstore]
    try:
        ds = xr.merge(dsets, join='exact')
        return ds
    except ValueError:
        return None

def open_delayed(df):
    """A dask.delayed wrapper around `open_dsets`.
    Allows us to open many datasets in parallel."""
    return dask.delayed(open_dsets)(df)

from collections import defaultdict

dsets = defaultdict(dict)
for group, df in cat.df.groupby(by=['source_id', 'experiment_id']):
    dsets[group[0]][group[1]] = open_delayed(df)

open_dsets(df)
dsets_ = dask.compute(dict(dsets))[0]

expt_da = xr.DataArray(expts, dims='experiment_id', name='experiment_id', coords={'experiment_id': expts})

dsets_aligned = {}

The main dictionary we will be working with is `dsets_`. Generally, `dsets_[source][scenario]` will be an xarray dataset with its data stored as a dask array.


In [15]:
dsets_

{'ACCESS-ESM1-5': {'historical': <xarray.Dataset>
  Dimensions:  (lat: 145, lon: 192, time: 1980)
  Coordinates:
      height   float64 ...
    * lat      (lat) float64 -90.0 -88.75 -87.5 -86.25 ... 86.25 87.5 88.75 90.0
    * lon      (lon) float64 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
    * time     (time) datetime64[ns] 1850-01-16T12:00:00 ... 2014-12-16T12:00:00
  Data variables:
      tas      (time, lat, lon) float32 dask.array<chunksize=(589, 145, 192), meta=np.ndarray>,
  'ssp126': <xarray.Dataset>
  Dimensions:  (lat: 145, lon: 192, time: 1032)
  Coordinates:
      height   float64 ...
    * lat      (lat) float64 -90.0 -88.75 -87.5 -86.25 ... 86.25 87.5 88.75 90.0
    * lon      (lon) float64 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
    * time     (time) datetime64[ns] 2015-01-16T12:00:00 ... 2100-12-16T12:00:00
  Data variables:
      tas      (time, lat, lon) float32 dask.array<chunksize=(516, 145, 192), meta=np.ndarray>,
  'ssp245': <xarray.Datas

A lot of code I used ended up needing the line `climatology = v['historical'].sel(time=slice('1981-01-01', '2010-12-31')).groupby('time.month').mean('time')` or something similar. The `sel`, `groupby`, and `mean` methods are all in xarray's documetation (linked at the start of this notebook). For `slice`, see [here](https://docs.python.org/3/library/functions.html#slice).

In [ ]:
for k, v in tqdm(dsets_.items()):

    # Check for any missing experiments that we expect
    expt_dsets = v.values()
    if any([d is None for d in expt_dsets]):
        print(f"Missing experiment for {k}")
        continue

    climatology = v['historical'].sel(time=slice('1981-01-01', '2010-12-31')).groupby('time.month').mean('time')

    for i in v:
        if i == 'historical':
            # When working with daily data, it's very easy for objects to run your computer out of memory, so we shorten historical runs because to the date range we need.
            anomaly = v[i].sel(time=slice('1981-01-01', '2010-12-31')).groupby('time.month') - climatology
        else:
            anomaly = v[i].groupby('time.month') - climatology
        # Because these files are too large to store in memory, we use the option compute=False to create a dask delayed object and then compute it later.
        # Remember to change the file location to the relevant folder.
        delayed_obj = anomaly.to_netcdf(path=f"~/tas-anomaly_{k}_{i}.nc", mode='w', compute=False, engine='netcdf4', format='NETCDF4')
        print(f"writing data to ~/tas-anomaly_{k}_{i}.nc")

        with progress.ProgressBar():
            results = delayed_obj.compute()